In [18]:
# Set up a chain rule to calculate the loss function @ saturation w/out propagating derivatives through saturation solver
# This was done in the Language of Molecules paper.

# 1_differentiable_saft works when no solvers are involved, i.e. for properties specified with a given (V, T)
# if, instead, we want to solve for saturation conditions, then I have the old problem of 

In [19]:
import Pkg; Pkg.activate(".")

using Revise
using Clapeyron
includet("./saftvrmienn.jl")
# These are functions we're going to overload for SAFTVRMieNN
import Clapeyron: a_res, saturation_pressure

using Flux
using Plots
using ForwardDiff, DiffResults

using Zygote, ChainRulesCore
using ImplicitDifferentiation

  Activating project at `~/SAFT_ML`


In [124]:
function make_NN_model(Mw, m, σ, λ_a, λ_r, ϵ)
    model = SAFTVRMieNN(
        params = SAFTVRMieNNParams(
            Mw=[Mw],
            segment=[m],
            sigma=[σ],
            lambda_a=[λ_a],
            lambda_r=[λ_r],
            epsilon=[ϵ],
            epsilon_assoc=Float32[],
            bondvol=Float32[],
        )
    )
    return model
end

# Hack to get around Clapeyron model construction API
function make_model(Mw, m, σ, λ_a, λ_r, ϵ)
    model = SAFTVRMie(["methane"])
    
    model.params.Mw[1] = Mw
    model.params.segment[1] = m
    model.params.sigma[1] = σ
    model.params.lambda_a[1] = λ_a
    model.params.lambda_r[1] = λ_r
    model.params.epsilon[1] = ϵ

    return model
end

function forward_sat_p(X)
    T = 100.0
    model = make_model(16.04, X...)

    p, Vₗ, Vᵥ = saturation_pressure(model, T)
    
    return [p, Vₗ, Vᵥ]
end

# Saturation conditions:
#* At conditions specified by y, these two conditions are satisfied
#? But how do I turn this into an optimality condition with the same
#? dimension as X
function conditions_sat_p(X, y)
    T = 100.0
    # Mw, m, σ, λ_a, λ_r, ϵ = X
    model = make_NN_model(16.04, X...)
    
    p, Vₗ, Vᵥ = y
    
    # @show p_sat = -(eos(model, Vᵥ, T) - eos(model, Vₗ, T))/(Vᵥ - Vₗ)
    @show p_sat = -(eos(model, Vᵥ, T) - eos(model, Vₗ, T))/(Vᵥ - Vₗ) - 17805.87602904105;
    
    obj = p - p_sat
    return obj
end

impl_saturation_pressure = ImplicitFunction(forward_sat_p, conditions_sat_p, linear_solver=DirectLinearSolver())

ImplicitFunction(forward_sat_p, conditions_sat_p, DirectLinearSolver(true), nothing)

In [125]:
m = make_model(16.04, X...)
m2 = make_NN_model(16.04, X...)
T = 100.0
@show p, Vₗ, Vᵥ = saturation_pressure(m, T);
@show p_sat = -(eos(m2, Vᵥ, T) - eos(m2, Vₗ, T))/(Vᵥ - Vₗ) - 17805.87602904105;
@show p - p_sat

(p, Vₗ, Vᵥ) = saturation_pressure(m, T) = (34113.7600622731, 3.6950451345479705e-5, 0.024068284363809818)
p_sat = -((eos(m2, Vᵥ, T) - eos(m2, Vₗ, T))) / (Vᵥ - Vₗ) - 17805.87602904105 = 34113.7600622731
p - p_sat = 0.0


0.0

In [134]:
@show a_res(m, Vᵥ, T, [1.0]);
@show a_res(m2, Vᵥ, T, [1.0]);
#! These don't match! This is really bad. There's a mistake somewhere.
#TODO: get dirty dev Clapeyron + Debugger

a_res(m, Vᵥ, T, [1.0]) = -0.012500723244825514


ErrorException: type Array has no field values

In [132]:
# X = [16.04, 1.0, 3.737e-10, 6.0, 12.504, 152.58]
X = [1.0, 3.737e-10, 6.0, 12.504, 152.58]
@show y = impl_saturation_pressure(X)
conditions_sat_p(X, y)

y = impl_saturation_pressure(X) = [34113.7600622731, 3.6950451345479705e-5, 0.024068284363809818]
p_sat = -((eos(model, Vᵥ, T) - eos(model, Vₗ, T))) / (Vᵥ - Vₗ) - 17805.87602904105 = 34113.7600622731


0.0

In [128]:
g = Zygote.gradient(X -> conditions_sat_p(X, y), X)

p_sat = -((eos(model, Vᵥ, T) - eos(model, Vₗ, T))) / (Vᵥ - Vₗ) - 17805.87602904105 = 34113.7600622731


([186112.62159890146, 1.811378731007528e13, -55953.40658916977, -10783.022852069103, 1888.3408548239495],)

In [129]:
g[1]

5-element Vector{Float64}:
 186112.62159890146
      1.811378731007528e13
 -55953.40658916977
 -10783.022852069103
   1888.3408548239495

In [47]:
model = SAFTVRMie(["methane"])
saturation_pressure(model, 100.0)

(34113.7600622731, 3.6950451345479705e-5, 0.024068284363809818)

In [131]:
Zygote.gradient(impl_saturation_pressure, X)
# ForwardDiff.gradient(impl_saturation_pressure, X)
#! This doesn't seem to have worked... :(

p_sat = -((eos(model, Vᵥ, T) - eos(model, Vₗ, T))) / (Vᵥ - Vₗ) - 17805.87602904105 = 34113.7600622731
p_sat = -((eos(model, Vᵥ, T) - eos(model, Vₗ, T))) / (Vᵥ - Vₗ) - 17805.87602904105 = 34113.7600622731


MethodError: MethodError: no method matching (::ProjectTo{Float64, NamedTuple{(), Tuple{}}})(::Vector{Float64})
Closest candidates are:
  (::ProjectTo)(!Matched::InplaceableThunk) at ~/.julia/packages/ChainRulesCore/0t04l/src/projection.jl:125
  (::ProjectTo{T})(!Matched::AbstractFloat) where T<:AbstractFloat at ~/.julia/packages/ChainRulesCore/0t04l/src/projection.jl:171
  (::ProjectTo{<:Real})(!Matched::Complex) at ~/.julia/packages/ChainRulesCore/0t04l/src/projection.jl:187
  ...

In [25]:
#* Instead, follow the example in Language of molecules paper, defining as the last newton step
